<a href="https://colab.research.google.com/github/Thiago-Reis-Porto/treinamento-h2ia/blob/main/Relat%C3%B3rio_Busca_com_heuristica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)


# Resolver o quebra-cabeças usando Buscas

In [29]:
import numpy as np

solution = np.array([[1,2,3],[4,5,6],[7,8,0]])

#---------------------------------------------------------------------------------------------------------
#--Classe do quebra-cabeça--------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------

class Puzzle:
  
  #Construtor recebe o pai e o estado(ou gera um estado aleatorio)
  def __init__(self, parent, state = np.random.choice([i for i in range(9)] , size=(3, 3), replace=False)):
    self.state = state #array com o estado
    self.blankPos = list(map(lambda x: x[0],(np.where(self.state == 0))) ) #posição do espaço
    self.isSolution = np.array_equal(solution,self.state) 
    self.g = self.__blank_Cost()
    self.h = self.__H_Cost()
    self.f = self.g + self.h
    self.visited = False
    self.children = [] #proximos movimentos possiveis
    self.parent = parent

#---------------------------------------------------------------------------------------------------------
#-Movimentos - -------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------
  #Move espaço para esquerda se possivel
  def __moveL(self):
    blankPos = self.blankPos
    if blankPos[1] == 0:
      return np.array([])
    newState = self.state.copy()
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]][blankPos[1]-1]
    newState[blankPos[0]][blankPos[1]-1] = 0
    return newState
  
  #Move espaço para direita se possivel
  def __moveR(self):
    blankPos = self.blankPos
    if blankPos[1] == 2:
      return np.array([])
    newState = self.state.copy()    
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]][blankPos[1]+1]
    newState[blankPos[0]][blankPos[1]+1] = 0
    return newState
  
  #Move espaço para cima se possivel
  def __moveU(self):
    blankPos = self.blankPos
    if blankPos[0] == 0:
      return np.array([])
    newState = self.state.copy()
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]-1][blankPos[1]]
    newState[blankPos[0]-1][blankPos[1]] = 0
    return newState
  #Move espaço para baixo se possivel
  def __moveD(self):
    blankPos = self.blankPos
    if blankPos[0] == 2:
      return np.array([])
    newState = self.state.copy()
    newState[blankPos[0]][blankPos[1]] = newState[blankPos[0]+1][blankPos[1]]
    newState[blankPos[0]+1][blankPos[1]] = 0
    return newState

#---------------------------------------------------------------------------------------------------------
#-----------Gera os proximos movimentos possiveis a partir do estado atual--------------------------------
#---------------------------------------------------------------------------------------------------------
  def genChildren(self):
    moves = [self.__moveR, self.__moveL, self.__moveU, self.__moveD]
    for i in moves:
      newMove = i()
      if newMove.any() and self.parent == None:
        self.children.append(Puzzle(self, newMove))
      
      elif newMove.any() and not np.array_equal(newMove, self.parent.state): 
        self.children.append(Puzzle(self, newMove))
#---------------------------------------------------------------------------------------------------------
#-------------------------------Metodos para heuristicas--------------------------------------------------
#---------------------------------------------------------------------------------------------------------
  def __H_Cost(self):
    cost = 0;
    for i in range(3):
      for j in range(3):
        if i==j==2:
          return cost
        if self.state[i][j] != solution[i][j]:
          cost+=1
        
  def __blank_Cost(self):  
      return 2 - self.blankPos[0] + 2 - self.blankPos[1]


## Busca A*

In [35]:
 
def aStar(puzzle):
  def isIn(state, nodes):
    for i in nodes:
      if np.array_equal(state, i.state):
        return True
    return False
  
  moves = 0
  if puzzle.isSolution:
    print("Movimentos =", moves)
    return puzzle
  
  frontier = { 'open':[], 'closed':[]}
  frontier['open'].append(puzzle)
  
  while True:
    
    puzzle.genChildren()
    
    frontier['open'].remove(puzzle)
    frontier['closed'].append(puzzle)
  
    
    for i in puzzle.children:
      if not isIn(i.state, frontier['open']) and not isIn(i.state, frontier['closed']):
        frontier['open'].append(i)
    
    puzzle = frontier['open'][0]
    for i in frontier['open']:
      if puzzle.h > i.h:
        puzzle = i

    #[ x for x in frontier['open'] if not (x==i).all()]
    moves+=1
    if puzzle.isSolution:
      print("Movimentos =", moves)
      return puzzle
    


In [36]:
p = Puzzle(None)
print(p.state)
s = aStar(p)


[[7 5 4]
 [2 0 3]
 [6 1 8]]
Movimentos = 469


## Discusta sobre o desempenho do método em questões de:


1.   Consumo de memória
2.   Processamento



**Memoria**: Não possui um custo tão alto, mas é nescessario guardar os estados(ou nós se for o caso) visitados, é mantida uma lista "aberta" com estados que devem ser expandidos, e a lista "fechada" dos que já foram expandidos.


**Processamento** : É um algoritimo bem eficiente, teve um desempenho melhor que dfs e bfs em geral, mas neste problema a minha implementação ainda demora bastante se o estado inicial prescisar de muitos movimentos, corre-se o risco do algoritimo se distanciar da solução.
